## 2. Attention

The purpose is towfold: (1) gain understanding of the attention mechanism, and (2) ensure that our implementation is correct by cross-checking with a built-in attention function.

* Taking an average
* Vectorizing our weighted average
* Learning weights for the wectorized weighted average
* Cross checking with a built-in attention function
* Expanding to mult-headed attention

### 2.1 Taking an Average

A naive method for predicting the next word would be to take an average of the word features that come before it. Let's look at the following example.

In [1]:
import torch
torch.manual_seed(103)

batch_size = 2
context_size = 3
n_embd = 4

x = torch.randint(high=10, size=(batch_size, context_size, n_embd), dtype=torch.float32)
print(f"Input ({batch_size=}, {context_size=}, {n_embd=}):\n {x}")

Input (batch_size=2, context_size=3, n_embd=4):
 tensor([[[5., 9., 1., 3.],
         [1., 8., 5., 2.],
         [5., 8., 5., 9.]],

        [[5., 9., 2., 5.],
         [4., 5., 8., 6.],
         [1., 7., 4., 4.]]])


We are going to therefore generate predictions for each position, so we are effectively getting multiple batches of examples from each sequence (as well as multiple independent sequences).

As a reminder: when you slice you don't get the value correspnding to the right integer e.g. `[:4]` it takes all elements up to but not including `4`.

In [2]:
y_output = torch.zeros((batch_size, context_size, n_embd))
for b in range(batch_size):
    for t in range(context_size):
        x_prev = x[b, : t + 1, :]
        y_output[b, t, :] = x_prev.mean(dim=0)
print(f"Output ({batch_size=}, {context_size=}):\n {y_output}")

Output (batch_size=2, context_size=3):
 tensor([[[5.0000, 9.0000, 1.0000, 3.0000],
         [3.0000, 8.5000, 3.0000, 2.5000],
         [3.6667, 8.3333, 3.6667, 4.6667]],

        [[5.0000, 9.0000, 2.0000, 5.0000],
         [4.5000, 7.0000, 5.0000, 5.5000],
         [3.3333, 7.0000, 4.6667, 5.0000]]])


### 2.2 Vectorizing our moving average

We can remove the for loop and vectorize the previous operation by the following technique: creating a square matrix of dimension `context_size`, then multiplying that matrix by our input matrix. 

And this is actually numerically equivalent to what we had before.

In [3]:
mask = torch.tril(torch.ones(context_size, context_size))
triangle_masked_weights = mask / mask.sum(dim=1, keepdim=True)
triangle_masked_weights

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

What we have is a square matrix: `(context_size, context_size)` multiplied by a `(context_size, n_embd)` matrix. 

And torch is going to broadcast this matrix operation across the batch dimension to give us ultimately what we want.

Which is a `(batch_size, context_size, n_embd)` matrix that is equivalent to what we had before.

In [4]:
y_output_2 = triangle_masked_weights @ x
print(f"My outputs are equivalent: {torch.allclose(y_output, y_output_2)}")
y_output_2

My outputs are equivalent: True


tensor([[[5.0000, 9.0000, 1.0000, 3.0000],
         [3.0000, 8.5000, 3.0000, 2.5000],
         [3.6667, 8.3333, 3.6667, 4.6667]],

        [[5.0000, 9.0000, 2.0000, 5.0000],
         [4.5000, 7.0000, 5.0000, 5.5000],
         [3.3333, 7.0000, 4.6667, 5.0000]]])

Why does this work? 
* The rows in the triangle matrix are being multiplied by the columns (which corresponds to the context/sequence length) in the input matrix. 
* The first row in the triangle matix zero's out the all but the very first element of the input matrix columns. Likewise, the second row in the triangle matrix zeros out all by the very first two elements of the input matrix columns. And so on and so forth.

What is happening here? 
* Well, this is a moving average with uniformly weighted across all prior positions of the input. 
* The first row has a 1, the second row has a 1/2, the third row has a 1/3, etc...
* But what if we wanted to take a weighted moving average, and ulimately learn the weights to use in the average? 
* This is what the attention mechanism does.

### 2.3 Self-Attention Mechanism

We are going to learn the weights to use by comparing the query and the key. Here we are using the terminology from the world of databases and hash tables. Your query is like your request. The key is the index of the data to be returned. And the values are what you ultimately are caring about. It doesn't map 1:1 in the world of LLMs, but this is the terminology used.
* Change 1: we are learning weights associated with projections of the input sequence.
* Change 2: we are creating a weight matrix that corresponds to the similarity of the query and the key. (Previously this was a uniform)
* Change 3: we are replacing the zeros with -infinity, because since e^-inf = 0

In [5]:
import math
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(243)

key_layer = nn.Linear(n_embd, n_embd)
query_layer = nn.Linear(n_embd, n_embd)
value_layer = nn.Linear(n_embd, n_embd)

q = key_layer(x)
k = query_layer(x)
v = value_layer(x)

weights = q @ k.transpose(-2, -1) * 1 / math.sqrt(k.shape[-1])
weights = weights.masked_fill(torch.tril(torch.ones(context_size, context_size)) == 0, float("-inf"))
weights = F.softmax(weights, dim=-1)
output = weights @ v
print(f"Input ({batch_size=}, {context_size=}, {n_embd=}):\n{v}")
print(f"Weights ({context_size=},{context_size=}) :\n{weights}")
print(f"Output ({batch_size=}, {context_size=}, {n_embd=}):\n{output}")

Input (batch_size=2, context_size=3, n_embd=4):
tensor([[[ 3.5171,  1.2840, -3.4459, -4.4998],
         [ 3.1850,  1.2937,  0.2993, -3.4091],
         [ 0.3842,  4.1011,  0.9978, -1.5259]],

        [[ 2.6459,  2.1178, -2.2431, -3.6423],
         [ 0.0575,  3.9665,  2.8881, -1.3670],
         [ 1.8560,  1.7649,  0.8492, -1.9648]]], grad_fn=<ViewBackward0>)
Weights (context_size=3,context_size=3) :
tensor([[[1.0000, 0.0000, 0.0000],
         [0.6018, 0.3982, 0.0000],
         [0.3446, 0.2355, 0.4198]],

        [[1.0000, 0.0000, 0.0000],
         [0.0133, 0.9867, 0.0000],
         [0.6703, 0.0380, 0.2918]]], grad_fn=<SoftmaxBackward0>)
Output (batch_size=2, context_size=3, n_embd=4):
tensor([[[ 3.5171,  1.2840, -3.4459, -4.4998],
         [ 3.3848,  1.2878, -1.9546, -4.0655],
         [ 2.1236,  2.4689, -0.6982, -2.9944]],

        [[ 2.6459,  2.1178, -2.2431, -3.6423],
         [ 0.0920,  3.9419,  2.8196, -1.3973],
         [ 2.3172,  2.0850, -1.1461, -3.0664]]], grad_fn=<UnsafeViewBac

* Change 4: We are applying a scaling factor of 1 / sqrt(d_k). This allows the softmax output to be more spread out when the values are large (the attention logit can become very large). 
* Here is an example.

In [6]:
print("no scaling:", F.softmax(torch.tensor([0, 1, 2, 3, 10, -1e4]).float(), dim=0))
print("with scaling:", F.softmax(torch.tensor([0, 1, 2, 3, 10, -1e4]).float() / n_embd, dim=0))

no scaling: tensor([4.5336e-05, 1.2324e-04, 3.3499e-04, 9.1059e-04, 9.9859e-01, 0.0000e+00])
with scaling: tensor([0.0548, 0.0704, 0.0904, 0.1161, 0.6682, 0.0000])


### 2.4 Checking our work

Let's to a cross-check using the pytorch attention function to make sure we're doing this correctly. This is a function, so it expects you to pass in the key, query, value data to it.

And we can see that we're getting the exact same results here.

In [7]:
expected_output = F.scaled_dot_product_attention(q, k, v, is_causal=True)

print(f"Output ({batch_size=}, {context_size=}, {n_embd=}):\n{expected_output}")
print(f"Your implementation is correct: {torch.allclose(output, expected_output)}")

Output (batch_size=2, context_size=3, n_embd=4):
tensor([[[ 3.5171,  1.2840, -3.4459, -4.4998],
         [ 3.3848,  1.2878, -1.9546, -4.0655],
         [ 2.1236,  2.4689, -0.6982, -2.9944]],

        [[ 2.6459,  2.1178, -2.2431, -3.6423],
         [ 0.0920,  3.9419,  2.8196, -1.3973],
         [ 2.3172,  2.0850, -1.1461, -3.0664]]], grad_fn=<UnsafeViewBackward0>)
Your implementation is correct: True


### 2.5 Multi-headed Attention

Now we can expand this to the setting of a multi-headed attention. The idea here is to have a bunch of these attention heads processing completely independently. We could create these independently, and then loop through them, and then concatenate them together. However, there's a slightly more efficient way of doing this. Let's make our input data slightly bigger to illustrate and use an example of 4 heads.

* It's not obvious until you draw it out, but we create our keys/queries/values together.

In [8]:
torch.manual_seed(103)

batch_size = 2
context_size = 3
n_embd = 12
n_heads = 4

x = torch.randint(high=10, size=(batch_size, context_size, n_embd), dtype=torch.float32)
print(f"Input ({batch_size=}, {context_size=}, {n_embd=}):\n {x}")

key_layer = nn.Linear(n_embd, n_embd)
query_layer = nn.Linear(n_embd, n_embd)
value_layer = nn.Linear(n_embd, n_embd)

q = key_layer(x)
k = query_layer(x)
v = value_layer(x)


Input (batch_size=2, context_size=3, n_embd=12):
 tensor([[[5., 9., 1., 3., 1., 8., 5., 2., 5., 8., 5., 9.],
         [5., 9., 2., 5., 4., 5., 8., 6., 1., 7., 4., 4.],
         [6., 9., 5., 3., 6., 8., 9., 0., 5., 2., 0., 6.]],

        [[7., 5., 5., 3., 6., 5., 3., 7., 4., 9., 0., 5.],
         [3., 9., 4., 7., 0., 3., 7., 1., 5., 5., 2., 6.],
         [2., 7., 5., 1., 0., 6., 2., 7., 4., 7., 3., 4.]]])


Then we can convert the head into a batch dimension, so now we have two independent batch dimensions.

The first corresponding to the batch of sequences, the second corresponding to the attention head.

So our final shape is going to be `(batch_size, n_heads, context_size, n_embd // n_heads)`

In [9]:
q = q.view(batch_size, context_size, n_heads, n_embd // n_heads).transpose(1, 2)
k = k.view(batch_size, context_size, n_heads, n_embd // n_heads).transpose(1, 2)
v = v.view(batch_size, context_size, n_heads, n_embd // n_heads).transpose(1, 2)

Then we can do the exact same thing as before.

In [10]:
torch.manual_seed(243)

key_layer = nn.Linear(n_embd, n_embd)
query_layer = nn.Linear(n_embd, n_embd)
value_layer = nn.Linear(n_embd, n_embd)

weights = q @ k.transpose(-2, -1) * 1 / math.sqrt(k.shape[-1])
weights = weights.masked_fill(torch.tril(torch.ones(context_size, context_size)) == 0, float("-inf"))
weights = F.softmax(weights, dim=-1)
output = weights @ v
print(f"Input ({v.shape}):\n{v}")
print(f"Weights ({weights.shape}) :\n{weights}")
print(f"Output ({output.shape=}):\n{output}")

Input (torch.Size([2, 4, 3, 3])):
tensor([[[[ 8.7784e+00, -4.8753e+00,  1.1932e+00],
          [ 5.0240e+00, -2.5953e-01, -1.5118e+00],
          [ 5.3240e+00, -1.2399e+00, -1.7522e+00]],

         [[-3.5147e-01, -1.9413e+00, -2.5365e+00],
          [-2.6749e-01, -1.2959e+00,  3.5526e-01],
          [-2.1949e-01, -2.6145e+00, -2.5183e+00]],

         [[ 5.9869e-01, -3.0716e+00,  4.2714e-01],
          [-2.2677e-01, -4.4387e+00,  7.7444e-01],
          [-1.5823e+00, -2.8861e+00,  7.8539e-01]],

         [[-5.0772e-01,  5.2677e+00, -2.1102e+00],
          [ 2.1462e-03,  3.1952e+00, -1.2774e+00],
          [-1.9456e-02,  3.5555e+00, -1.9040e+00]]],


        [[[ 4.9868e+00,  9.7068e-01, -2.1595e+00],
          [ 8.2696e+00, -3.3180e+00,  4.1589e-01],
          [ 6.1282e+00, -2.8202e+00, -6.9385e-01]],

         [[ 2.5676e+00,  1.4350e+00,  6.6007e-03],
          [-6.6369e-01, -4.2443e-01, -1.8966e+00],
          [ 1.8987e+00,  2.0759e+00,  5.6721e-01]],

         [[ 9.5145e-01, -3.8799e+0

Just as before, we can perform our sanity check to ensure we did it right.

In [14]:
expected_output = F.scaled_dot_product_attention(q, k, v, is_causal=True)

print(f"Output ({batch_size=}, {n_heads=}, {context_size=}, {n_embd=}): {expected_output.shape}")
print(f"Your implementation is correct: {torch.allclose(output, expected_output)}")

Output (batch_size=2, n_heads=4, context_size=3, n_embd=12): torch.Size([2, 4, 3, 3])
Your implementation is correct: True
